In [2]:
import openai
import json
import os
import time
import numpy as np
import pandas as pd

openai.api_key = "your openai api key" 

temperatures = [0]
studies = ["kim2018"]
models = ["gpt-4-0125-preview"]#, "gpt-4"]
system_prompt = "You are presented with a partial specification of a visualization. Your task is to complete the design in an optimal manner so that people can read the visualization quickly and accurately. You rely upon visualization design best practices and principles from graphical perception research to make your recommendation."

def get_task_type(path, study, name):
    file_path = f'{path}/{study}/{name}'

    # Read the content of the file
    with open(file_path, 'r') as file:
        content = file.read()

    # Check for the presence of "summary" and "value"
    contains_summary = "summary" in content
    contains_value = "value" in content
    
    if contains_summary and contains_value:
        raise ValueError("Both 'summary' and 'value' are present, which is not allowed.")
    elif not contains_summary and not contains_value:
        raise ValueError("Neither 'summary' nor 'value' is present, which is required.")

    # Return the keyword that is present
    if contains_summary:
        return "summary"
    elif contains_value:
        return "value"


    
def generate_responses(directory, study, filename, max_retries=3):
    for model in models:
        for temp in temperatures:
            response_file = f'./gpt_responses/{study}_responses/{model}_{temp}/{os.path.splitext(filename)[0]}_{model}_{temp}.txt'
            
            if not os.path.exists(f'./gpt_responses/{study}_responses/{model}_{temp}/'):
                os.makedirs(f'./gpt_responses/{study}_responses/{model}_{temp}/')
            
            if os.path.exists(response_file):
                print(response_file + " exists!!!\n")
                continue
            
            
            
            print("*" * 100)
            print(model + " " + str(temp) + " " + study + " " + filename)

            task_type = get_task_type(directory, study, filename)
            
            prompt = ""
            with open('./prompt_templates/{}'.format(task_type) + '_begin.txt', 'r') as file:
                prompt = file.read()
                
            with open(os.path.join(directory + '/{}/'.format(study), filename), 'r') as file:
                content = file.read()
                prompt += "\n\n" + content.strip() + "\n\n"

            with open('./prompt_templates/{}'.format(task_type) + '_end.txt', 'r') as file:
                content = file.read()
                prompt += content.strip()

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ]
    
            attempt = 0
            while attempt < max_retries:
                
                try:
                    completion = openai.ChatCompletion.create(
                        model=model,
                        temperature=temp,
                        seed = 42,
                        messages=messages
                    )
                    messages.append({"role": "assistant", "content": completion.choices[0].message.content})
                    print(completion.choices[0].message.content)

                    with open(response_file, 'w') as file:
                        for message in messages:
                            for key, value in message.items():
                                file.write(f"{key}: {value}\n")
                            file.write('-' * 40)
                            file.write('\n')
                    print('write complete')
                    break  

                except Exception as e:
                    print(f"Error encountered: {e}. Retrying...")
                    attempt += 1
                    time.sleep(20)

            if attempt == max_retries:
                print(f"Failed to process {response_file} after {max_retries} attempts.")
                return "failed"
            return



dir_path = './common_partial_spec'
i = 0
for study in studies:
    for filename in os.listdir(dir_path + '/{}'.format(study)):
        if filename.endswith(".txt"):
            file_path = os.path.join(dir_path + '/{}/'.format(study), filename)
            response = generate_responses(dir_path, study, filename)
            if response == "failed":
                time.sleep(30)
                response = generate_responses(dir_path, study, filename)
            print(i)
            i+=1
            
        
            
    


./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/1053_gpt-4-0125-preview_0.txt exists!!!

0
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/1047_gpt-4-0125-preview_0.txt exists!!!

1
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/289_gpt-4-0125-preview_0.txt exists!!!

2
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/504_gpt-4-0125-preview_0.txt exists!!!

3
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/1090_gpt-4-0125-preview_0.txt exists!!!

4
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/262_gpt-4-0125-preview_0.txt exists!!!

5
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/276_gpt-4-0125-preview_0.txt exists!!!

6
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/1084_gpt-4-0125-preview_0.txt exists!!!

7
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/510_gpt-4-0125-preview_0.txt exists!!!

8
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/538_gpt-4-0125-preview_0.txt exists!!!

9
./gpt_responses/kim2018_re

```json
{
  "chart": {
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "mark": "point",
    "encoding": {
      "x": {"field": "q1", "type": "quantitative"},
      "color": {"field": "q2", "type": "quantitative", "scale": {"zero": true}},
      "shape": {"field": "n", "type": "nominal"}
    }
  },
  "reason": "Using 'x' to encode 'q1' allows for easy comparison of its values across categories, 'color' for 'q2' respects the partial specification and adds depth without clutter, and 'shape' for 'n' distinguishes categories clearly without aggregation."
}
```
write complete
203
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/1040_gpt-4-0125-preview_0.txt exists!!!

204
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/1068_gpt-4-0125-preview_0.txt exists!!!

205
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/298_gpt-4-0125-preview_0.txt exists!!!

206
./gpt_responses/kim2018_responses/gpt-4-0125-preview_0/1056_gpt-4-0125-preview_0.txt exists!!!

20

In [10]:
temperatures = [0]
studies = ["kim2018"]
models = ["gpt-3.5-turbo-0125"]
system_prompt = "You are presented with a partial specification for a visualization. Your task is to complete this design in an optimal manner so that people can read the visualization quickly and accurately. You rely upon visualization design best practices and principles from graphical perception research to make your recommendation. Ensure that your completion fully respects the provided partial specification."


def altair_generate_responses(directory, study, filename, max_retries=3):
    for model in models:
        for temp in temperatures:
            response_file = f'./gpt_responses/{study}_responses/altair_{model}_{temp}/{os.path.splitext(filename)[0]}_{model}_{temp}.txt'
            
            if not os.path.exists(f'./gpt_responses/{study}_responses/altair_{model}_{temp}/'):
                os.makedirs(f'./gpt_responses/{study}_responses/altair_{model}_{temp}/')
            
            if os.path.exists(response_file):
                print(response_file + " exists!!!\n")
                continue
            
            
            
            print("*" * 100)
            print(model + " " + str(temp) + " " + study + " " + filename)

            task_type = get_task_type(directory, study, filename)
            
            prompt = ""
            with open('./prompt_templates_altair/{}'.format(task_type) + '_begin.txt', 'r') as file:
                prompt = file.read()
                
            with open(os.path.join(directory + '/{}/'.format(study), filename), 'r') as file:
                content = file.read()
                prompt += "\n\n" + content.strip() + "\n\n"

            with open('./prompt_templates_altair/{}'.format(task_type) + '_end.txt', 'r') as file:
                content = file.read()
                prompt += content.strip()

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ]
    
            attempt = 0
            while attempt < max_retries:
                
                try:
                    completion = openai.ChatCompletion.create(
                        model=model,
                        temperature=temp,
                        seed = 42,
                        messages=messages
                    )
                    messages.append({"role": "assistant", "content": completion.choices[0].message.content})
                    print(completion.choices[0].message.content)

                    with open(response_file, 'w') as file:
                        for message in messages:
                            for key, value in message.items():
                                file.write(f"{key}: {value}\n")
                            file.write('-' * 40)
                            file.write('\n')
                    print('write complete')
                    break  

                except Exception as e:
                    print(f"Error encountered: {e}. Retrying...")
                    attempt += 1
                    time.sleep(20)

            if attempt == max_retries:
                print(f"Failed to process {response_file} after {max_retries} attempts.")
                return "failed"
            return



dir_path = './common_partial_spec'
i = 0
for study in studies:
    for filename in os.listdir(dir_path + '/{}'.format(study)):
        if filename.endswith(".txt"):
            file_path = os.path.join(dir_path + '/{}/'.format(study), filename)
            response = altair_generate_responses(dir_path, study, filename)
            if response == "failed":
                time.sleep(30)
                response = altair_generate_responses(dir_path, study, filename)
            print(i)
            i+=1
            
            

****************************************************************************************************
gpt-3.5-turbo-0125 0 kim2018 1053.txt
{
    "chart": {
        "mark": {"type": "point"},
        "encoding": {
            "x": {"field": "n", "type": "ordinal"},
            "y": {"field": "q2", "type": "quantitative"},
            "size": {"field": "q1", "type": "quantitative"}
        },
        "scale": {"y": {"type": "linear", "zero": true}}
    },
    "reason": "Using point marks to show individual data points for q2 on the y-axis, encoding q1 with point size to allow easy comparison of aggregate properties of individual visual marks."
}
write complete
0
****************************************************************************************************
gpt-3.5-turbo-0125 0 kim2018 1047.txt
{
    "chart": {
        "mark": "point",
        "encoding": {
            "x": {"field": "q1", "type": "quantitative"},
            "y": {"field": "q2", "type": "quantitative"},
            

{
    "chart": {
        "mark": {
            "type": "point"
        },
        "encoding": {
            "x": {"field": "q2", "type": "quantitative"},
            "y": {"field": "n", "type": "nominal"},
            "color": {"field": "q1", "type": "quantitative"}
        },
        "scale": {
            "x": {"type": "linear", "zero": true},
            "y": {"type": "ordinal"}
        }
    },
    "reason": "Using color encoding for q1 allows readers to easily retrieve and compare individual q1 values while maintaining the required three encoding channels."
}
write complete
13
****************************************************************************************************
gpt-3.5-turbo-0125 0 kim2018 712.txt
{
    "chart": {
        "mark": {
            "type": "point"
        },
        "encoding": {
            "x": {"field": "q1", "type": "quantitative"},
            "y": {"field": "q2", "type": "quantitative"},
            "color": {"field": "n", "type": "nominal"}
      

{
    "chart": {
        "mark": {
            "type": "point",
            "size": {"field": "size", "type": "quantitative", "scale": {"zero": true}}
        },
        "encoding": {
            "x": {"field": "q1", "type": "quantitative"},
            "y": {"field": "q2", "type": "quantitative"},
            "size": {"field": "size", "type": "quantitative"}
        }
    },
    "reason": "Using point marks with size encoding for q1 and q2 allows for easy comparison of individual data points while the size channel provides an additional quantitative dimension for comparison."
}
write complete
26
****************************************************************************************************
gpt-3.5-turbo-0125 0 kim2018 882.txt
{
    "chart": {
        "mark": "point",
        "encoding": {
            "x": {"field": "q2", "type": "quantitative"},
            "y": {"field": "n", "type": "nominal"},
            "size": {"field": "q1", "type": "quantitative"}
        },
        "sele

{
    "chart": {
        "mark": {"type": "point"},
        "encoding": {
            "x": {"field": "q1", "type": "quantitative"},
            "y": {"field": "q2", "type": "quantitative"},
            "color": {"field": "n", "type": "nominal"}
        },
        "selection": {"highlight": {"type": "single", "empty": "none", "on": "mouseover"}}
    },
    "reason": "This completion respects the partial specification by using exactly three encoding channels, one for each variable, and allows readers to easily compare individual data points for the summary task involving q1 and n."
}
write complete
38
****************************************************************************************************
gpt-3.5-turbo-0125 0 kim2018 317.txt
Error encountered: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600). Retrying...
{
    "chart": {
        "mark": {"type": "point"},
        "encoding": {
            "x": {"field": "n", "type": "

{
    "chart": {
        "mark": "point",
        "encoding": {
            "x": {"field": "n", "type": "nominal"},
            "y": {"field": "q1", "type": "quantitative"},
            "color": {"field": "q2", "type": "quantitative"}
        },
        "selection": {},
        "view": {"height": 300, "width": 400}
    },
    "reason": "Using a scatter plot with q1 on the y-axis, n on the x-axis, and q2 encoded by color allows for easy comparison of q1 values across categories in n without data aggregation."
}
write complete
50
****************************************************************************************************
gpt-3.5-turbo-0125 0 kim2018 854.txt
{
    "chart": {
        "mark": {
            "type": "point"
        },
        "encoding": {
            "x": {"field": "q1", "type": "quantitative", "scale": {"zero": true}},
            "y": {"field": "n", "type": "ordinal"},
            "size": {"field": "q2", "type": "quantitative", "scale": {"zero": true}}
        }
  

...

{
    "chart": {
        "mark": "point",
        "encoding": {
            "x": {"field": "q1", "type": "quantitative"},
            "y": {"field": "q2", "type": "quantitative"},
            "row": {"field": "n", "type": "nominal"}
        },
        "facet": {
            "row": {"field": "n", "type": "nominal"}
        }
    },
    "reason": "This completion respects the given specification by using exactly three encoding channels for q1, q2, and n, while allowing readers to easily perform value tasks involving q1 by comparing individual data points."
}
write complete
1092
****************************************************************************************************
gpt-3.5-turbo-0125 0 kim2018 31.txt
{
    "chart": {
        "chart": alt.Chart(data).mark_point().encode(
            x=alt.X('q2:Q', scale=alt.Scale(zero=True)),
            y='n:N',
            color='q1:Q'
        )
    },
    "reason": "Using a scatter plot with q2 on the x-axis, n on the y-axis, and encoding 

{
    "chart": {
        "mark": "point",
        "encoding": {
            "x": {"field": "q1", "type": "quantitative"},
            "y": {"field": "q2", "type": "quantitative"},
            "color": {"field": "n", "type": "nominal"}
        },
        "selection": {},
        "view": {"height": 300, "width": 400},
        "selection": {}
    },
    "reason": "Using x for q1, y for q2, and color for n respects the partial specification, allowing readers to easily perform value tasks involving q1 by comparing individual data points."
}
write complete
1105
****************************************************************************************************
gpt-3.5-turbo-0125 0 kim2018 1103.txt
{
    "chart": {
        "chart": alt.Chart(data).mark_point().encode(
            x=alt.X('q1:Q', scale=alt.Scale(zero=True)),
            color=alt.Color('q2:Q', scale=alt.Scale(zero=True)),
            y=alt.Y('n:O', scale=alt.Scale(zero=False))
        ),
        "reason": "Using x for q1, colo

{
    "chart": {
        "mark": "point",
        "encoding": {
            "x": {"field": "q2", "type": "quantitative"},
            "y": {"field": "n", "type": "nominal"},
            "size": {"field": "q1", "type": "quantitative"}
        },
        "height": 300,
        "width": 400
    },
    "reason": "Using point marks to show individual data points, encoding q1 with size allows for easy comparison of quantitative values while maintaining the ability to read individual values for q1 and n."
}
write complete
1118
****************************************************************************************************
gpt-3.5-turbo-0125 0 kim2018 469.txt
{
    "chart": {
        "mark": {"type": "point"},
        "encoding": {
            "x": {"field": "q1", "type": "quantitative", "scale": {"zero": true}},
            "y": {"field": "n", "type": "ordinal"},
            "size": {"field": "q2", "type": "quantitative", "scale": {"zero": true}}
        }
    },
    "reason": "Using x fo

{
    "chart": {
        "mark": {"type": "point"},
        "encoding": {
            "x": {"field": "q1", "type": "quantitative"},
            "y": {"field": "q2", "type": "quantitative"},
            "color": {"field": "n", "type": "nominal", "scale": {"type": "linear", "zero": true}}
        }
    },
    "reason": "Using a scatter plot with q1 and q2 on the axes and color encoding for n allows for easy comparison of aggregate properties of individual data points without data aggregation."
}
write complete
1131
****************************************************************************************************
gpt-3.5-turbo-0125 0 kim2018 694.txt
{
    "chart": {
        "mark": {"type": "point"},
        "encoding": {
            "x": {"field": "q1", "type": "quantitative", "scale": {"type": "linear", "zero": true}},
            "y": {"field": "q2", "type": "quantitative"},
            "color": {"field": "n", "type": "nominal"}
        }
    },
    "reason": "Using x for q1, y for q

{
    "chart": {
        "mark": {"type": "point"},
        "encoding": {
            "x": {"field": "q1", "type": "quantitative"},
            "y": {"field": "q2", "type": "quantitative"},
            "color": {"field": "n", "type": "nominal"}
        },
        "selection": {}
    },
    "reason": "Using a scatter plot with q1 on the x-axis, q2 on the y-axis, and color encoding for n allows for easy comparison of individual data points across categories, supporting the summary task effectively."
}
write complete
1144
****************************************************************************************************
gpt-3.5-turbo-0125 0 kim2018 521.txt
{
    "chart": {
        "mark": "point",
        "encoding": {
            "x": {"field": "q1", "type": "quantitative"},
            "y": {"field": "q2", "type": "quantitative"},
            "row": {"field": "n", "type": "nominal"}
        },
        "facet": {"row": {"field": "n", "type": "nominal"}}
    },
    "reason": "This comple